# Milestone 2: Model Evaluation & Ranking Generation
**Mục tiêu:**
1. Đọc dữ liệu `labels.json` (chứa input BibTeX và Ground Truth).
2. Tái cấu trúc dữ liệu theo từng bài báo (Paper-based grouping).
3. Sử dụng mô hình đã huấn luyện để xếp hạng (Ranking) các reference candidates.
4. Tính chỉ số **MRR (Mean Reciprocal Rank)**.
5. Xuất file `pred.json` theo đúng định dạng yêu cầu nộp bài.

**Yêu cầu đầu vào:**
* `labels.json`: File dữ liệu test.
* `models/best_matcher.pkl`: Model đã train.
* `models/feature_names.pkl`: List tên features tương ứng.


In [ ]:
import pandas as pd
import numpy as np
import json
import os
import sys
import joblib
from tqdm import tqdm
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Setup path để import src module
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import từ src.ml module
from src.ml import (
    load_json,
    save_json,
    normalize_text_basic,
    get_tokens,
    safe_year_diff,
    compute_pairwise_features,
    compute_tfidf_cosine_single,
    parse_bibtex_smart,
    clean_latex,
    normalize_id,
    transform_to_paper_based
)

pd.set_option('display.max_columns', None)
print("✅ Libraries và src.ml modules imported successfully!")

Libraries imported successfully!


In [65]:
# --- CẤU HÌNH ĐƯỜNG DẪN ---
# Hãy thay đổi đường dẫn phù hợp với môi trường của bạn
TEST_FILE_PATH = '../../dataset_final/test/labels.json'           # File dữ liệu test
MODEL_PATH = '../../dataset_final/models/best_matcher.pkl'   # File model (đã train ở bước trước)
FEATURE_NAME_PATH = '../../dataset_final/models/feature_names.pkl' # Danh sách features
OUTPUT_DIR = 'submission_output'         # Thư mục chứa kết quả pred.json

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created output directory: {OUTPUT_DIR}")


In [ ]:
# Các hàm helper đã được import từ src.ml:
# - normalize_text_basic(text) -> str
# - get_tokens(text_or_list) -> set
# - safe_year_diff(y1, y2) -> int

print("📚 Helper functions đã import từ src.ml.features:")
print("   - normalize_text_basic")
print("   - get_tokens")  
print("   - safe_year_diff")

In [ ]:
# BibTeX Parser đã được import từ src.ml.bibtex_parser:
# - parse_bibtex_smart(bib_string) -> dict
# - clean_latex(text) -> str
# - normalize_id(text) -> str

print("📚 BibTeX Parser đã import từ src.ml.bibtex_parser:")
print("   - parse_bibtex_smart: Parser V3 (Title, Authors, Year, ID)")
print("   - clean_latex: Xử lý LaTeX markup")
print("   - normalize_id: Chuẩn hóa DOI/arXiv")

## 1. Feature Engineering
**Lưu ý quan trọng:** Các hàm dưới đây phải **giống hệt** logic bạn đã sử dụng khi huấn luyện mô hình (Training Phase). Nếu thay đổi, model sẽ nhận diện sai đặc trưng.


In [ ]:
# Helper functions đã import từ module
# Không cần define lại

def normalize_text(text):
    """Wrapper for normalize_text_basic."""
    return normalize_text_basic(text)

In [ ]:
from fuzzywuzzy import fuzz

# Sử dụng compute_pairwise_features từ module
# Thêm TF-IDF cosine cho single pairs

def compute_features_with_tfidf(row):
    """Compute features bao gồm TF-IDF cosine."""
    feats = compute_pairwise_features(row)
    
    # Thêm TF-IDF cosine cho single pair
    q_tit = normalize_text_basic(row.get('bib_title', ''))
    c_tit = normalize_text_basic(row.get('cand_title', ''))
    feats['feat_title_tfidf_cosine'] = compute_tfidf_cosine_single(q_tit, c_tit)
    
    return feats

print("✅ Feature engineering function ready (sử dụng src.ml.features)")

## 2. Load & Transform Data
Chúng ta cần chuyển đổi dữ liệu từ danh sách phẳng (flat list) trong `manual.json` sang cấu trúc **Paper-based**.
* Mỗi `source_paper_id` sẽ là một nhóm.
* Tập **Candidates** của nhóm đó là tập hợp tất cả các `ground_truth` unique xuất hiện trong paper.


In [ ]:
# --- DATA LOADING & TRANSFORMATION ---
try:
    raw_data = load_json(TEST_FILE_PATH)
    print(f"✅ Loaded {len(raw_data)} entries.")
except FileNotFoundError:
    print("❌ Error: File not found.")
    raw_data = []

# Transform sang Paper-based structure sử dụng module
papers_db = transform_to_paper_based(raw_data, parse_bibtex_smart)

print(f"✅ Data transformed into {len(papers_db)} papers.")

Loaded 812 entries.
✅ Data transformed into 39 papers.


## 3. Load Trained Model
Load model SVM/RandomForest/XGBoost đã lưu từ bước Training.


In [71]:
try:
    model = joblib.load(MODEL_PATH)
    feature_names = joblib.load(FEATURE_NAME_PATH)
    print(f"✅ Model loaded: {type(model).__name__}")
    print(f"📝 Expected features: {len(feature_names)}")
except FileNotFoundError:
    print("⚠️ WARNING: Model file not found. Code will run with RANDOM SCORES for demonstration.")
    model = None
    feature_names = []


✅ Model loaded: XGBClassifier
📝 Expected features: 13


## 4. Ranking Pipeline & Prediction
Với mỗi bài báo:
1. Tạo cặp (Query, Candidate) cho tất cả ứng viên.
2. Tính feature.
3. Dự đoán xác suất match.
4. Lấy Top 5 ứng viên có điểm cao nhất.


In [ ]:
global_mrr_sum = 0
global_query_count = 0

print("🚀 Starting Ranking Pipeline...")

for paper_id, data in papers_db.items():
    queries = data['queries']
    candidates_dict = data['candidates']
    candidates_list = list(candidates_dict.values())
    
    if not candidates_list:
        continue

    # Init Output Structure
    submission_data = {
        "partition": "test", 
        "groundtruth": {},
        "prediction": {}
    }
    
    # Loop qua từng query trong paper
    for query in tqdm(queries, desc=f"Paper {paper_id}", leave=False):
        bib_key = query['key']
        true_id = query['true_id']
        
        # Groundtruth
        submission_data['groundtruth'][bib_key] = true_id
        
        # Pairing & Feature Calc
        pairs = []
        for cand in candidates_list:
            row = {}
            row.update(query)
            row.update(cand)
            pairs.append(row)
            
        # Compute Features sử dụng hàm từ module
        feats_list = [compute_features_with_tfidf(p) for p in pairs]
        df_feats = pd.DataFrame(feats_list)
        
        # Predict
        scores = []
        if model:
            # Ensure columns exist
            for col in feature_names:
                if col not in df_feats.columns: 
                    df_feats[col] = 0.0
            
            X_input = df_feats[feature_names]
            scores = model.predict_proba(X_input)[:, 1]
        else:
            scores = np.random.rand(len(pairs))
            
        # Ranking
        ranked_candidates = []
        for idx, score in enumerate(scores):
            ranked_candidates.append((candidates_list[idx]['cand_id'], score))
        
        ranked_candidates.sort(key=lambda x: x[1], reverse=True)
        top_5 = [x[0] for x in ranked_candidates[:5]]
        
        # Save Prediction
        submission_data['prediction'][bib_key] = top_5
        
        # Calc MRR
        if true_id in top_5:
            rank = top_5.index(true_id) + 1
            global_mrr_sum += 1.0 / rank
        else:
            global_mrr_sum += 0.0
            
        global_query_count += 1
        
    # Save JSON Output sử dụng module
    safe_pid = str(paper_id).replace('/', '_')
    save_path = os.path.join(OUTPUT_DIR, f"{safe_pid}_pred.json")
    save_json(submission_data, save_path)

print("✅ Ranking completed for all papers.")

🚀 Starting Ranking Pipeline...


✅ Ranking completed for all papers.


## 5. Evaluation Report (Nhận xét)


In [73]:
final_mrr = global_mrr_sum / global_query_count if global_query_count > 0 else 0

print("="*40)
print("📊 FINAL EVALUATION REPORT")
print("="*40)
print(f"Papers Processed: {len(papers_db)}")
print(f"Total Queries:    {global_query_count}")
print(f"Metric MRR:       {final_mrr:.4f}")
print("="*40)

if final_mrr > 0.8:
    print("🌟 Nhận xét: Xuất sắc. Model xếp hạng reference rất chính xác.")
elif final_mrr > 0.5:
    print("👍 Nhận xét: Khá. Model đưa ra ứng viên đúng trong top 5 thường xuyên.")
else:
    print("⚠️ Nhận xét: Cần cải thiện. Kiểm tra lại Features hoặc Model.")

print(f"\n📁 File kết quả (pred.json) lưu tại: {os.path.abspath(OUTPUT_DIR)}")


📊 FINAL EVALUATION REPORT
Papers Processed: 39
Total Queries:    812
Metric MRR:       0.8349
🌟 Nhận xét: Xuất sắc. Model xếp hạng reference rất chính xác.

📁 File kết quả (pred.json) lưu tại: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\notebooks\submission_output
